# Excercises: Exploratory Analysis
> * Do your work for this exercise in a notebook named explore. 
> * Use the techniques in the lesson to explore the store item demand dataset and the opsd dataset.
> * For the store item demand data, you will need to choose a method of aggregating such that each observation is a unique date. 
> * For both datasets you should choose a method of splitting your data and only explore the training split.

# Store Item Demand Data

Do your work for this exercise in a notebook named explore. Use the techniques in the lesson to explore the store item demand dataset and the opsd dataset.

For the store item demand data, you will need to choose a method of aggregating such that each observation is a unique date. For both datasets you should choose a method of splitting your data and only explore the training split.

OPSD Data